# Tutorial 1. The Measurement Control 

This tutorial covers basic usage of quantify focusing on running basic experiments using `MeasurementControl`. 
The `MeasurementControl` is the main `Instrument` in charge of running any experiment. It takes care of saving the data in a standardized format as well as live plotting of the data during the experiment. 
`quantify` makes a distinction between `soft`(ware) controlled measurements and `hard`(ware) controlled measurements. 

In a `soft` measurement `MeasurementControl` is in charge of the measurement loop and consecutively sets and gets datapoints. A `soft` measurement can be 1D, 2D or higher dimensional and also supports adaptive measurements in which the datapoints are determined during the measurement loop. 

In a `hard` measurement the hardware (such as an AWG or a central controller) is in charge of the measurement loop. In this case, the datapoints to be acquired are determined before the experiment starts and are precompiled into the hardware which is then armed and starts acquisition. In a `hard` measurement `MeasurementControl` does not take care of the measurement loop but still takes care of the data storage and live plotting of the experiment. 

In [1]:
import numpy as np 
import xarray as xr 
import matplotlib.pyplot as plt 
from qcodes import ManualParameter, Parameter
%matplotlib inline

In [2]:
from quantify.measurement import MeasurementControl
import quantify.visualization.pyqt_plotmon as pqm





In [3]:
MC = MeasurementControl('MC')
plotmon = pqm.PlotMonitor_pyqt('plotmon')
MC.instr_plotmon(plotmon.name)

In [4]:
MC.instr_plotmon.get_instr().tuid()

'latest'

# Define a simple model 

In [5]:
from time import sleep

In [6]:

def CosFunc(t, amplitude, frequency, phase, offset):
    """A simple cosine function"""
    return amplitude * np.cos(2 * np.pi * frequency * t + phase) + offset

# Parameters are created to emulate a system being measured 
amp = ManualParameter('amp', initial_value=1, unit='V', label='Amplitude')
freq = ManualParameter('freq', initial_value=.5, unit='Hz', label='Frequency')
t = ManualParameter('t', initial_value=1, unit='s', label='Time')
phi = ManualParameter('phi', initial_value=0, unit='Rad', label='Phase')

acq_delay = ManualParameter('acq_delay', initial_value=.1, unit='s')

def cosine_model():
    sleep(acq_delay())
    return CosFunc(t(), amp(), freq(), phase=phi(), offset=0)

# We wrap our function in a Parameter to be able to give 
sig = Parameter(name='sig', label='Signal level', unit='V', get_cmd=cosine_model)

In [7]:
acq_delay(0.0)

# A 1D soft(ware) controlled loop

In [8]:
acq_delay(0.0) # we set this to a non-zero value to see the live plotting in action. 

In [9]:
MC.set_setpars(t)
MC.set_setpoints(np.linspace(0, 5, 50))
MC.set_getpars(sig)
dset = MC.run('Cosine test')


 100% completed 	elapsed time: 0.4s 	time left: 0.0s


In [10]:
dset.attrs

{'tuid': '20200504-221748-47d6db', '2D-grid': False}

In [11]:

# By default the MC updates the datafile and live plot every 0.1 seconds to reduce overhead. 
# the total overhead is ~0.05s per update
MC.update_interval(0.01) # Setting it even to 0.01 makes a dramatic difference, try it out!

In [12]:
MC.set_setpars(t)
MC.set_setpoints(np.linspace(0, 50, 1000))
MC.set_getpars(sig)
dset = MC.run('Cosine test')



 100% completed 	elapsed time: 0.9s 	time left: 0.0s


# A 2D soft(ware) controlled loop

It is often desired to measure heatmaps (2D grids) of some parameter. 
This can be done by specifying two set parameters. 
The setpoints of the grid can be specified in two ways. 

## Method 1, a quick grid. 

In [13]:
MC.update_interval(.1)

In [14]:
times = np.linspace(0, 5, 500)
amps = np.linspace(-1, 1, 31)

MC.set_setpars([t, amp])
MC.set_setpoints(times)
MC.set_setpoints_2D(amps)
MC.set_getpars(sig)
dset = MC.run('2D Cosine test')


 100% completed 	elapsed time: 6.9s 	time left: 0.0s6s


## Method 2, custom tuples in 2D 

.. note:: 

    it is also possible to do this for higher dimensional loops

In [15]:
acq_delay(0.001)

In [16]:
MC.update_interval()

0.1

In [17]:
r = np.linspace(0, 1.5, 2000)
dt = np.linspace(0, 1, 2000)

f = 10

theta = np.cos(2*np.pi*f*dt)
def polar_coords(r, theta):

    x = r*np.cos(2*np.pi*theta)
    y = r*np.sin(2*np.pi*theta)
    return x, y 

x,y = polar_coords(r, theta)
setpoints = np.column_stack([x,y])

In [18]:

MC.instr_plotmon('plotmon')

In [19]:
MC.update_interval(0.5)

In [40]:

MC.set_setpars([t, amp])
MC.set_setpoints(setpoints)
MC.set_getpars(sig)
dset = MC.run('2D radial setpoints')


 100% completed 	elapsed time: 6.7s 	time left: 0.0ss


# Below is for testing

In [39]:

# This snippet exists to quickly reload and test the live plotting. 
plotmon.close()
from importlib import reload
reload(pqm)
plotmon = pqm.PlotMonitor_pyqt('plotmon')